In [1]:
import requests
import pandas as pd
import gfwapiclient as gfw
import os
import asyncio
from shapely.geometry import box, mapping

In [2]:
access_token = "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImtpZEtleSJ9.eyJkYXRhIjp7Im5hbW" \
"UiOiJXaWxsIFBvbmN6YWsiLCJ1c2VySWQiOjQ2NzE1LCJhcHBsaWNhdGlvbk5hbWUiOiJXaWxsIFBvbmN6YWsi" \
"LCJpZCI6MjgwNywidHlwZSI6InVzZXItYXBwbGljYXRpb24ifSwiaWF0IjoxNzQ4OTU5MjIwLCJleHAiOjIwN" \
"jQzMTkyMjAsImF1ZCI6ImdmdyIsImlzcyI6ImdmdyJ9.i9JoOMrqnRuQks9RwcBJpbohYXvby7UigXgImksE6pD" \
"6FVZXuC-2-rZBoPCxNpQTJAtVDtgf5VunVxvs0Eelry88PyLRtzeXqRrE4YjH_wVGPuk4ZrXz5c9a4PnTt8xbglq" \
"ohIXha403c37UCDIPeV4Z7y8e3e6WxCWkaiW-5kD0Z-23CIB9ZC7gWh01fL64HqaQruCOIyRD_hRJB1PUdy0t9J" \
"dwk56lqcipcjvQVLZXvInzk-twhzIefaWZuLLBbdSOfWtlt6-NBqdaidCQfe6QChoHDa3lFUNMHCcQMOe_geYsj" \
"ObvXvuX6R0_4CmWq7E4t6w52_y0SjLKvVTScyz95RBm7xQMjmrFsx4VsHwpLNv9i_DtUKQuVt_OHGA6DwgIdtV0r" \
"sOKyPYosH4OLELMNjPuEPi9VC6HR-lagVTMnsGNCofsoOzPlbJNVQkaE2K2hFnD9rWAJF1F1efj9PmPznsahB9Jo" \
"km0Mw8lOiU74EkKFtYKIsXam2E2QOs3"

In [13]:
gfw_client = gfw.Client(access_token=access_token)

In [ ]:
# DATASET API

In [14]:
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [-180.0, -85.0511287798066],
            [-180.0, 0.0],
            [0.0, 0.0],
            [0.0, -85.0511287798066],
            [-180.0, -85.0511287798066],
        ]
    ],
}

In [15]:
async def main():
   
    sar_infrastructure_result = await gfw_client.datasets.get_sar_fixed_infrastructure(
        geometry=geometry
    )

    sar_infrastructure_df = sar_infrastructure_result.df()
    print(sar_infrastructure_df.info())
    print(sar_infrastructure_df.head())



In [17]:
sar_infrastructure_df = await main()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   structure_id          1121 non-null   int64              
 1   lat                   1121 non-null   float64            
 2   lon                   1121 non-null   float64            
 3   label                 1121 non-null   object             
 4   structure_start_date  1121 non-null   datetime64[ns, UTC]
 5   structure_end_date    829 non-null    datetime64[ns, UTC]
 6   label_confidence      1121 non-null   object             
dtypes: datetime64[ns, UTC](2), float64(2), int64(1), object(2)
memory usage: 61.4+ KB
None
   structure_id       lat        lon    label      structure_start_date  \
0        170796  1.509645 -78.888106  unknown 2021-11-01 00:00:00+00:00   
1        254126  1.029763 -79.745796      oil 2017-01-01 00:00:00+00:00   
2        3

In [ ]:
# VESSELS API

In [18]:
vessel_search_result = await gfw_client.vessels.search_vessels(
    where="ssvid='775998121' AND shipname='DON TITO'",
    datasets=["public-global-vessel-identity:latest"],
    includes=["MATCH_CRITERIA", "OWNERSHIP"],
)

In [19]:
vessel_search_df = vessel_search_result.df()
print(vessel_search_df.info())
print(vessel_search_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   dataset                         2 non-null      object
 1   registry_info_total_records     2 non-null      int64 
 2   registry_info                   2 non-null      object
 3   registry_owners                 2 non-null      object
 4   registry_public_authorizations  0 non-null      object
 5   combined_sources_info           2 non-null      object
 6   self_reported_info              2 non-null      object
 7   matchCriteria                   2 non-null      object
dtypes: int64(1), object(7)
memory usage: 260.0+ bytes
None
                              dataset  registry_info_total_records  \
0  public-global-vessel-identity:v3.0                            0   
1  public-global-vessel-identity:v3.0                            0   

  registry_info registry_owne

In [ ]:
# 4WINGS API

In [23]:
async def wings_main():
    report_result = await gfw_client.fourwings.create_report(
        spatial_resolution="LOW",
        temporal_resolution="MONTHLY",
        group_by="GEARTYPE",
        datasets=["public-global-fishing-effort:latest"],
        start_date="2022-01-01",
        end_date="2022-05-01",
        region={
            "dataset": "public-eez-areas",
            "id": "5690",
        },
    )
    report_df = report_result.df()
    print(report_df.info())
    print(report_df.head(5))
    return report_df

report_df = await wings_main()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41916 entries, 0 to 41915
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   date                     41916 non-null  object 
 1   detections               0 non-null      object 
 2   flag                     0 non-null      object 
 3   gear_type                41916 non-null  object 
 4   hours                    41916 non-null  float64
 5   vessel_ids               41916 non-null  int64  
 6   vessel_id                0 non-null      object 
 7   vessel_type              0 non-null      object 
 8   entry_timestamp          0 non-null      object 
 9   exit_timestamp           0 non-null      object 
 10  first_transmission_date  0 non-null      object 
 11  last_transmission_date   0 non-null      object 
 12  imo                      0 non-null      object 
 13  mmsi                     0 non-null      object 
 14  call_sign             